# install

In [1]:
%pip install opencv-python
%pip install matplotlib
%pip install numpy
%pip install albumentations
%pip install ultralytics==8.0.230
%pip install gradio
%pip install torch
%pip install ruamel
%pip install easyocr

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement ruamel (from versions: none)
ERROR: No matching distribution found for ruamel


Note: you may need to restart the kernel to use updated packages.


# Code

In [2]:
import cv2 
from ultralytics import YOLO 
import gradio as gr
import pandas as pd
import easyocr
import numpy as np
import pathlib

e:\Brad\.conda\lib\site-packages\torch\cuda\__init__.py:749: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ..\c10\cuda\CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
e:\Brad\.conda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# windows path fix
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

In [4]:
# Define constants
BOX_COLORS = {
    "unchecked": (242, 48, 48),
    "checked": (38, 115, 101),
    "block": (242, 159, 5)
}
BOX_PADDING = 2


In [5]:
# Load models
DETECTION_MODEL = YOLO("../models/detector-model.pt") 
CLASSIFICATION_MODEL = YOLO("../models/classifier-model.pt")

In [6]:
def isInside(bbox_a, bbox_b):
    bbox_b = list(bbox_b)
    bbox_a = bbox_a[0]
    # Check if X and Y coordinates of bbox_a are greater than or equal to the X and Y coordinates of bbox_b
    if bbox_a[0] >= bbox_b[0] and bbox_a[1] >= bbox_b[1]:
        # Check if X coordinate of bbox_a is less than (X coordinate of bbox_b + width)
        if bbox_a[2] < bbox_b[0] + bbox_b[2]:
            # Check if Y coordinate of bbox_a is less than (Y coordinate of bbox_b + height)
            if bbox_a[3] < bbox_b[1] + bbox_b[3]:
                return True
    return False

In [7]:
def is_bbox_inside(xywh, bbox):
    x, y = xywh[0], xywh[1]
    w, h = xywh[2], xywh[3]

    # Check if all corner points of the inner bbox are inside the outer bbox
    for point in bbox:
        px, py = point
        if px < x or px > x + w or py < y or py > y + h:
            return False

    return True

In [8]:
def getContour(image,bbox):
    # Load image, grayscale, Gaussian blur, Otsu's threshold
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (7,7), 0)
    thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

    # Create rectangular structuring element and dilate
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    dilate = cv2.dilate(thresh, kernel, iterations=4)

    # Find contours and draw rectangle
    cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]

    for i in cnts:
        if isInside(bbox,cv2.boundingRect(i)):
            return(i)
    raise Exception('WTF?!!')

In [13]:
image_path = '../src/images/image2_mod.jpg'
image = cv2.imread(image_path)

In [14]:
# Predict on image
results = DETECTION_MODEL.predict(source=image, conf=0.01, iou=0.8) # Predict on image
boxes = results[0].boxes # Get bounding boxes


0: 640x512 5 uncheckeds, 4 checkeds, 3 msts, 424.5ms
Speed: 4.5ms preprocess, 424.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 512)


In [15]:
reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory
result = reader.readtext(image)

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


In [16]:
DATA = []
for i,box in enumerate(boxes):

    # Get start and end points of the current box
    start_box = (int(box.xyxy[0][0]), int(box.xyxy[0][1]))
    end_box = (int(box.xyxy[0][2]), int(box.xyxy[0][3]))
    ibox = image[start_box[1]:end_box[1], start_box[0]: end_box[0], :]

    probs = CLASSIFICATION_MODEL.predict(source=ibox, conf=0.5)[0].probs
    classification_class = list(BOX_COLORS)[2 - int(probs.top1)] 
    classification_class_conf = round(probs.top1conf.item(), 2)
    cls = classification_class if classification_class_conf > 0.9 else list(BOX_COLORS)[int(box.cls)]

    #centerpoint
    bbox = box.xyxy
    x = int(bbox[:,0] + bbox[:,2]) / 2
    y = int(bbox[:,1] + bbox[:,3]) / 2
    bbox = bbox.int().tolist()
    
    text = []
    text_bboxes = []

    if cls == 'checked':
        cnt = getContour(image,bbox)
        for detection in result:
            bbox = detection[0]
            if is_bbox_inside(cv2.boundingRect(cnt),bbox):
                text.append(detection[1])  # Append text to list
                text_bboxes.append(bbox)

        # Find the bounding box closest to the point (x, y)
        closest_bbox = None
        closest_distance = float('inf')
        index = 0
        for i,bbox in enumerate(text_bboxes):
            centroid_x = sum([p[0] for p in bbox]) / 4
            centroid_y = sum([p[1] for p in bbox]) / 4
            distance = np.sqrt((centroid_x - x)**2 + (centroid_y - y)**2)
            if distance < closest_distance and centroid_y<y+15 and centroid_y>y-15 :
                closest_bbox = bbox
                closest_distance = distance
                index = i

        # Draw bounding box on the image
        if closest_bbox is not None:
            bbox = [[int(p[0]), int(p[1])] for p in closest_bbox]  # Convert float coordinates to int
            for i in range(4):
                cv2.line(image, tuple(bbox[i]), tuple(bbox[(i + 1) % 4]), (255, 255, 0), 2)
            print(text_bboxes)
            DATA.append({'class': cls, 'checkbox_coords': [x, y], 'closest_text': text[index]})
        cv2.circle(image, (int(x), int(y)), 5, (0, 255, 0), -1)

cv2.imwrite('test.jpg',image)



0: 32x32 unchecked 1.00, block 0.00, checked 0.00, 6.1ms
Speed: 0.0ms preprocess, 6.1ms inference, 1.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 unchecked 1.00, checked 0.00, block 0.00, 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 checked 0.86, block 0.14, unchecked 0.00, 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)

0: 32x32 checked 0.84, block 0.16, unchecked 0.00, 6.3ms
Speed: 1.0ms preprocess, 6.3ms inference, 0.0ms postprocess per image at shape (1, 3, 32, 32)
[[[703, 1065], [737, 1065], [737, 1081], [703, 1081]], [[789, 1081], [861, 1081], [861, 1099], [789, 1099]], [[543.2928932188134, 1202.2928932188136], [575.8755776840268, 1199.5169226570886], [576.7071067811866, 1213.7071067811864], [545.1244223159732, 1216.4830773429114]]]

0: 32x32 unchecked 0.99, checked 0.00, block 0.00, 5.7ms
Speed: 1.0ms preprocess, 5.7ms inference, 0.0ms postprocess

True